In [130]:
import pandas as pd

In [131]:
# File names
ministry_file = 'bundeshaushalt_all_ministries_2012_2026.xlsx'
subcategory_file = 'bundeshaushalt_subcategories_2012_2026.xlsx'

# Load both datasets
ministry_df = pd.read_excel(ministry_file, sheet_name='All_Data')
print(f"{len(ministry_df)} ministry-level records")

subcat_df = pd.read_excel(subcategory_file, sheet_name='All_Data')
print(f"{len(subcat_df)} subcategory records")

354 ministry-level records
45 subcategory records


In [132]:
# Remove irrelevant columns in both datasets
columns_to_remove = ['Budgetnummer', 'Relativer_Wert_zu_Eltern_Prozent']
ministry_df = ministry_df.drop(columns=columns_to_remove)
subcat_df = subcat_df.drop(columns=columns_to_remove)

In [133]:
# Convert ID column to numeric (removes leading zeros)
ministry_df['ID'] = pd.to_numeric(ministry_df['ID'], errors='coerce')
subcat_df['ID'] = pd.to_numeric(subcat_df['ID'], errors='coerce')

In [134]:
# Calculate total federal budget per year (sum of all ministries)
years = sorted(ministry_df['Jahr'].unique())
total_rows = []

for year in years:
    ministry_year = ministry_df[ministry_df['Jahr'] == year]
    total_budget = ministry_year['Wert_Euro'].sum()

    total_row = {
        'Jahr': year,
        'ID': '0',
        'Kategorie': 'Total Federal Budget',
        'Wert_Euro': total_budget,
        'Relativer_Wert_Prozent': 100.0,
    }
    total_rows.append(total_row)
    print(f"  Year {year}: {total_budget:,.0f} €")

# Create DataFrame with totals
totals_df = pd.DataFrame(total_rows)

  Year 2012: 311,600,000,000 €
  Year 2013: 310,000,000,000 €
  Year 2014: 296,500,000,000 €
  Year 2015: 306,900,000,000 €
  Year 2016: 316,900,000,000 €
  Year 2017: 329,100,000,000 €
  Year 2018: 343,600,000,000 €
  Year 2019: 356,400,000,000 €
  Year 2020: 508,529,758,000 €
  Year 2021: 572,725,714,000 €
  Year 2022: 495,791,475,000 €
  Year 2023: 461,211,782,000 €
  Year 2024: 476,807,656,000 €
  Year 2025: 502,546,135,000 €
  Year 2026: 524,540,138,000 €


In [135]:
# Calculate "Other Federal Budget" (exclude the 5 main ministries of interest and for comparison using their IDs -> Labour & Social = 11, Health = 15, Defense = 14, Education = 17, Debt Servicing = 32)
excluded_ids = [11, 14, 15, 17, 32]
other_ministries_df = ministry_df[~ministry_df['ID'].isin(excluded_ids)]

years = sorted(other_ministries_df['Jahr'].unique())
total_other_rows = []

for year in years:
    other_year = other_ministries_df[other_ministries_df['Jahr'] == year]
    total_other = other_year['Wert_Euro'].sum()

    total_other_row = {
        'Jahr': year,
        'ID': '99',
        'Kategorie': 'Other Federal Budget',
        'Wert_Euro': total_other,
        'Relativer_Wert_Prozent': 0.0,
    }
    total_other_rows.append(total_other_row)
    print(f"  Year {year}: {total_other:,.0f} €")

total_other_df = pd.DataFrame(total_other_rows)

  Year 2012: 99,202,131,000 €
  Year 2013: 105,660,877,000 €
  Year 2014: 94,521,374,000 €
  Year 2015: 104,454,130,000 €
  Year 2016: 107,319,465,000 €
  Year 2017: 109,839,254,000 €
  Year 2018: 121,053,335,000 €
  Year 2019: 123,778,198,000 €
  Year 2020: 220,590,747,000 €
  Year 2021: 282,498,612,000 €
  Year 2022: 188,885,372,000 €
  Year 2023: 166,281,729,000 €
  Year 2024: 179,026,607,000 €
  Year 2025: 182,244,023,000 €
  Year 2026: 172,424,460,000 €


In [136]:
# Calculate "Other Labour & Social" (Ministry of Labour & Social = 11 minus subcategory 1102 = pension insurance)
other_11_rows = []
for year in years:
    # Get Labour & Social ministry total
    labour_social = ministry_df[(ministry_df['Jahr'] == year) & (ministry_df['ID'] == 11)]['Wert_Euro'].values
    labour_social_value = labour_social[0] if len(labour_social) > 0 else 0

    # Get Rentenversicherung subcategory
    renten = subcat_df[(subcat_df['Jahr'] == year) & (subcat_df['ID'] == 1102)]['Wert_Euro'].values
    renten_value = renten[0] if len(renten) > 0 else 0

    # Calculate difference
    other_value = labour_social_value - renten_value

    other_11_row = {
        'Jahr': year,
        'ID': 98,
        'Kategorie': 'Other Labour & Social Expenses',
        'Wert_Euro': other_value,
        'Relativer_Wert_Prozent': None
    }
    other_11_rows.append(other_11_row)
    print(f"  Year {year}: {other_value:,.0f} €")

other_11_df = pd.DataFrame(other_11_rows)

  Year 2012: 125,776,138,000 €
  Year 2013: 118,874,642,000 €
  Year 2014: 33,550,830,000 €
  Year 2015: 35,943,656,000 €
  Year 2016: 36,617,812,000 €
  Year 2017: 39,330,170,000 €
  Year 2018: 39,037,233,000 €
  Year 2019: 39,931,302,000 €
  Year 2020: 60,780,426,000 €
  Year 2021: 50,251,075,000 €
  Year 2022: 44,295,820,000 €
  Year 2023: 45,179,870,000 €
  Year 2024: 48,374,468,000 €
  Year 2025: 55,918,513,000 €
  Year 2026: 57,331,406,000 €


In [137]:
# Calculate "Other Health" (Health Ministry = 15 without subcategories 1501 = statutory health insurance and 1502 = nursing insurance )
other_15_rows = []

for year in years:
    # Get Health ministry total
    health = ministry_df[(ministry_df['Jahr'] == year) & (ministry_df['ID'] == 15)]['Wert_Euro'].values
    health_value = health[0] if len(health) > 0 else 0

    # Get Gesetzliche Krankenversicherung subcategory
    kranken = subcat_df[(subcat_df['Jahr'] == year) & (subcat_df['ID'] == 1501)]['Wert_Euro'].values
    kranken_value = kranken[0] if len(kranken) > 0 else 0

    # Get Pflegevorsorge subcategory
    pflege = subcat_df[(subcat_df['Jahr'] == year) & (subcat_df['ID'] == 1502)]['Wert_Euro'].values
    pflege_value = pflege[0] if len(pflege) > 0 else 0

    # Calculate difference
    other_value = health_value - kranken_value - pflege_value

    other_15_row = {
        'Jahr': year,
        'ID': 97,
        'Kategorie': 'Other Health Expenses',
        'Wert_Euro': other_value,
        'Relativer_Wert_Prozent': None
    }
    other_15_rows.append(other_15_row)
    print(f"  Year {year}: {other_value:,.0f} €")

other_15_df = pd.DataFrame(other_15_rows)

  Year 2012: 302,815,000 €
  Year 2013: 302,376,000 €
  Year 2014: 322,895,000 €
  Year 2015: 310,239,000 €
  Year 2016: 523,526,000 €
  Year 2017: 599,820,000 €
  Year 2018: 639,066,000 €
  Year 2019: 727,282,000 €
  Year 2020: 9,867,819,000 €
  Year 2021: 14,326,428,000 €
  Year 2022: 10,843,991,000 €
  Year 2023: 4,650,719,000 €
  Year 2024: 1,978,337,000 €
  Year 2025: 1,901,592,000 €
  Year 2026: 1,692,896,000 €


In [138]:
# Build final_df by combining all needed datasets

# Get specific ministries from ministry_df
ministry_ids = [14, 17, 32]
selected_ministries = ministry_df[ministry_df['ID'].isin(ministry_ids)].copy()

# Get specific subcategories from subcat_df
subcategory_ids = [1102, 1501, 1502]
selected_subcats = subcat_df[subcat_df['ID'].isin(subcategory_ids)].copy()

# Combine all dataframes
final_df = pd.concat([
    other_15_df,
    other_11_df,
    total_other_df,
    totals_df,
    selected_ministries,
    selected_subcats
], ignore_index=True)

print(f"  ✓ Final dataset has {len(final_df)} total records")

# Sort by year and ID
final_df = final_df.sort_values(['Jahr', 'ID'])
print(final_df.groupby('Kategorie')['Wert_Euro'].count())

  ✓ Final dataset has 150 total records
Kategorie
14 Bundesministerium der Verteidigung                                     15
17 Bundesministerium für Bildung, Familie, Senioren, Frauen und Jugend     2
17 Bundesministerium für Familie, Senioren, Frauen und Jugend             13
32 Bundesschuld                                                           15
Gesetzliche Krankenversicherung                                           15
Other Federal Budget                                                      15
Other Health Expenses                                                     15
Other Labour & Social Expenses                                            15
Pflegevorsorge                                                            15
Rentenversicherung und Grundsicherung                                     15
Total Federal Budget                                                      15
Name: Wert_Euro, dtype: int64


/tmp/ipython-input-3421982095.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([


In [139]:
# Translate German categories to English and clean up
translation_map = {
    '14 Bundesministerium der Verteidigung': 'Ministry of Defense',
    '17 Bundesministerium für Bildung, Familie, Senioren, Frauen und Jugend': 'Ministry of Education & Family',
    '17 Bundesministerium für Familie, Senioren, Frauen und Jugend': 'Ministry of Education & Family',
    '32 Bundesschuld': 'Federal Debt Service',
    'Gesetzliche Krankenversicherung': 'Subsidies Statutory Health Insurance',
    'Pflegevorsorge': 'Subsidies Long-term Care Insurance',
    'Rentenversicherung und Grundsicherung': 'Subsidies Pension Insurance',
}

final_df['Kategorie'] = final_df['Kategorie'].replace(translation_map)

print(f"  ✓ Translated {len(translation_map)} categories")

print("\nFinal_df summary after translation:")
print(final_df.groupby('Kategorie')['Wert_Euro'].count())

  ✓ Translated 7 categories

Final_df summary after translation:
Kategorie
Federal Debt Service                    15
Ministry of Defense                     15
Ministry of Education & Family          15
Other Federal Budget                    15
Other Health Expenses                   15
Other Labour & Social Expenses          15
Subsidies Long-term Care Insurance      15
Subsidies Pension Insurance             15
Subsidies Statutory Health Insurance    15
Total Federal Budget                    15
Name: Wert_Euro, dtype: int64


In [140]:
# Calculate Relativer_Wert_Prozent (percentage share of total budget)
for year in final_df['Jahr'].unique():
    # Get total budget for this year
    total_budget = final_df[(final_df['Jahr'] == year) & (final_df['Kategorie'] == 'Total Federal Budget')]['Wert_Euro'].values[0]

    # Calculate percentage for all rows in this year
    year_mask = final_df['Jahr'] == year
    final_df.loc[year_mask, 'Relativer_Wert_Prozent'] = (final_df.loc[year_mask, 'Wert_Euro'] / total_budget) * 100

print(final_df[final_df['Jahr'] == 2012][['Jahr', 'Kategorie', 'Wert_Euro', 'Relativer_Wert_Prozent']])

     Jahr                             Kategorie     Wert_Euro  \
60   2012                   Ministry of Defense   31871857000   
61   2012        Ministry of Education & Family    7370220000   
62   2012                  Federal Debt Service   32539470000   
0    2012                 Other Health Expenses     302815000   
15   2012        Other Labour & Social Expenses  125776138000   
107  2012           Subsidies Pension Insurance     354802000   
105  2012  Subsidies Statutory Health Insurance      71587000   
106  2012    Subsidies Long-term Care Insurance   14110980000   
45   2012                  Total Federal Budget  311600000000   
30   2012                  Other Federal Budget   99202131000   

     Relativer_Wert_Prozent  
60                10.228452  
61                 2.365282  
62                10.442705  
0                  0.097181  
15                40.364614  
107                0.113865  
105                0.022974  
106                4.528556  
45            

In [143]:
# ADDITIONAL Check if percentage shares sum to 100% for each year (excluding Total Federal Budget) -> ensures that the stacked bar chart works after all!
print("Sum of percentage shares by year (excluding Total Federal Budget):")
print()

for year in sorted(final_df['Jahr'].unique()):
    year_data = final_df[(final_df['Jahr'] == year) & (final_df['ID'] != '0')]
    total_share = year_data['Relativer_Wert_Prozent'].sum()
    print(f"Year {year}: {total_share:.2f}%")

print()
print("All years should sum to approximately 100%")

Sum of percentage shares by year (excluding Total Federal Budget):

Year 2012: 100.00%
Year 2013: 100.00%
Year 2014: 100.00%
Year 2015: 100.00%
Year 2016: 100.00%
Year 2017: 100.00%
Year 2018: 100.00%
Year 2019: 100.00%
Year 2020: 100.00%
Year 2021: 100.00%
Year 2022: 100.00%
Year 2023: 100.00%
Year 2024: 100.00%
Year 2025: 100.00%
Year 2026: 100.00%

All years should sum to approximately 100%


In [144]:
final_df.head()

,Jahr,ID,Kategorie,Wert_Euro,Relativer_Wert_Prozent
60,2012,14,Ministry of Defense,31871857000,10.228452
61,2012,17,Ministry of Education & Family,7370220000,2.365282
62,2012,32,Federal Debt Service,32539470000,10.442705
0,2012,97,Other Health Expenses,302815000,0.097181
15,2012,98,Other Labour & Social Expenses,125776138000,40.364614


In [142]:
# Export final_df to CSV
output_file = 'federal_budget_final_data.csv'
final_df.to_csv(output_file, index=False)
print(f"  Total records: {len(final_df)}")

  Total records: 150
